# Value for Care

In [7]:
import os
import sys
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
import ipywidgets as widgets
from IPython.display import clear_output 


%matplotlib inline
%pylab inline
matplotlib.rcParams['figure.figsize'] = (10, 6)

spark = SparkSession \
    .builder \
    .appName("Value of Care") \
    .getOrCreate()

df = spark.read.load("/Users/seankeefe/Desktop/Payment_and_Value_of_Care_Hospital.csv",format="csv", sep=",", inferSchema="true", header="true")
#ReqData = "/Users/seankeefe/Desktop/Hospital_Revised_Flatfiles/Payment_and_Value_of_Care-Hospital.csv"
#df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(ReqData)

df.registerTempTable('Data')

df2 = spark.read.load("/Users/seankeefe/Desktop/Payment-National.csv",format="csv", sep=",", inferSchema="true", header="true")
#NationalRate = "/Users/seankeefe/Desktop/Hospital_Revised_Flatfiles/Payment-National.csv"
#df2 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(NationalRate)

df2.registerTempTable('Master')


category= "SELECT Measure_Name from Master"
categoryName = spark.sql(category)


state= "SELECT DISTINCT State from Data order by State"
stateName = spark.sql(state)

_dataList = categoryName.collect()
_dataListState = stateName.collect()
# print(_dataList)
_category =[]
_selectedCat = ["NA"]
_category.append("NA")
for i in range(len(_dataList)):
    _category.append(_dataList[i][0])

    
_state =[]
_selectedSta = ["All"]
_state.append("All")
for i in range(len(_dataListState)):
    _state.append(_dataListState[i][0])    
    
selectState = widgets.Dropdown(
    options=_state,
    description='State:',
    disabled=False
)   
display(selectState)
select = widgets.Dropdown(
    options=_category,
    description='Category:',
    disabled=False
)
display(select)
    

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        _selectedCat[0] = (change['new'])    
 
def on_change_state(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        _selectedSta[0] = (change['new'])   

select.observe(on_change)
selectState.observe(on_change_state)
clear_output()




# Payment and Value care

In [8]:
print("Ploting for: "+ _selectedSta[0] +", " + _selectedCat[0])

if _selectedCat[0] == "Payment for heart failure patients":
   stateRate = 15959
elif _selectedCat[0] == "Payment for heart attack patients":
   stateRate = 22760 
else:
   stateRate = 14817

# print(t)
# print(_selectedSta[0])

query = " SELECT Hospital_Name,Payment,Value_of_care_category,M.National_payment as th"
query += " ,(CASE WHEN Value_of_care_category LIKE 'Average mortality%' then " +str(stateRate) 
query += " WHEN Value_of_care_category LIKE 'Better mortality%' then " +str(stateRate + 1000) 
query += " WHEN Value_of_care_category LIKE 'Worse mortality%' then " +str(stateRate - 1000) +" ELSE NULL END) AS Mor" 
query += " FROM Data D JOIN Master M"
query += " ON D.Payment_measure_ID = M.Measure_ID"
query += " WHERE Payment != 'Not Available'"


if _selectedSta[0] != "All":
    query += " AND State = '" +  _selectedSta[0] + "'" 
    

query += " AND Payment_measure_name = '" +  _selectedCat[0] + "'"
query += " ORDER by D.Hospital_Name"

# print(query)
distint = spark.sql(query)
#  distint.show();



_x=[]
_y=[]
_text=[]
_th = []
_Mor =[]
_dataList = distint.collect()
for i in range(len(_dataList)):
    _x.append(_dataList[i][0])
    _y.append(_dataList[i][1])
    _th.append(_dataList[i][3])
    _text.append(_dataList[i][2])
    _Mor.append(_dataList[i][4])
    
# print(_selectedCat[0])

import plotly
# Set Plotly credetials

plotly.tools.set_credentials_file(username='keefeseanc', api_key='A8CaFbFKVZZjBQLnT4QI')
import plotly.plotly as py
import plotly.tools as plotly_tools
import plotly.graph_objs as go
from plotly.graph_objs import *
 #Sign in the plotly API
py.sign_in("keefeseanc", "A8CaFbFKVZZjBQLnT4QI")

Payment = go.Scatter(
    x=_x,
    y=_y,
    name = 'Payment',
    text = _text
)

Mort = go.Scatter(
    x=_x,
    y=_Mor,
    name = 'Mortality',
    hoverinfo='none'
)

National_Rate = go.Scatter(
    x=_x,
    y=_th,
    name = 'National Rate'
)

data = [Payment,National_Rate,Mort]
py.iplot(data, filename='basic-line')

Ploting for: All, NA


AnalysisException: "cannot resolve '`M.Measure_ID`' given input columns: [D.Measure start date, D.State, D.Lower estimate, D.Hospital_Name, M.Measure_Name, D.Measure end date, D.Value_of_care_category, D.Higher estimate, M.National payment, D.Denominator, D.Provider ID, D.Phone number, D.Value of care display ID, D.Value of care display name, D.Payment measure name, D.Payment footnote, D.ZIP Code, M.Measure end date, D.Address, M.Number greater than national payment, M.Measure start date, M.Footnote, M.Number less than national payment, M.Payment_measure_ID, D.County name, D.Value of care footnote, D.Payment, D.Payment category, M.Number same as national payment, D.City, M.Number of hospitals too few, D.Payment_measure_ID]; line 1 pos 351;\n'Sort ['D.Hospital_Name ASC NULLS FIRST], true\n+- 'Project ['Hospital_Name, 'Payment, 'Value_of_care_category, 'M.National_payment AS th#186, CASE WHEN 'Value_of_care_category LIKE Average mortality% THEN 14817 WHEN 'Value_of_care_category LIKE Bet